<a href="https://colab.research.google.com/github/NguyenMai2001/YOLOV3/blob/master/convetYolov3Format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd

In [ ]:
os.chdir("/content/drive/MyDrive/COLAB/YOLOv3/images_dataset/csv_folder")
classes_data = pd.read_csv("class-descriptions-boxable.csv", header=None)
classes_data.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [ ]:
print(classes_data[1])

0        Tortoise
1       Container
2          Magpie
3      Sea turtle
4        Football
          ...    
596    Wheelchair
597    Rugby ball
598     Armadillo
599       Maracas
600        Helmet
Name: 1, Length: 601, dtype: object


In [ ]:
#define class
classes = ['Person']

class_strings = []
for i in classes:
  req_classes = classes_data.loc[classes_data[1]==i]
  string = req_classes.iloc[0][0]
  print(string)
  class_strings.append(string)

print(class_strings)

/m/01g317
['/m/01g317']


In [ ]:
annotation_data = pd.read_csv("train-annotations-bbox.csv",
                              usecols=['ImageID',
                                       'LabelName',
                                       'XMin',
                                       'XMax',
                                       'YMin',
                                       'YMax'])

annotation_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938


In [ ]:
#get label matching with class
filtered_class_data = annotation_data.loc[annotation_data['LabelName'].isin(class_strings)].copy()
filtered_class_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938


In [ ]:
#add new columns for Yolo format

filtered_class_data['classNumber'] = ''
filtered_class_data['center x'] = ''
filtered_class_data['center y'] = ''
filtered_class_data['width'] = ''
filtered_class_data['height'] = ''

filtered_class_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax,classNumber,center x,center y,width,height
0,000002b66c9c498e,/m/01g317,0.012500,0.195312,0.148438,0.587500,,,,,
1,000002b66c9c498e,/m/01g317,0.025000,0.276563,0.714063,0.948438,,,,,
2,000002b66c9c498e,/m/01g317,0.151562,0.310937,0.198437,0.590625,,,,,
3,000002b66c9c498e,/m/01g317,0.256250,0.429688,0.651563,0.925000,,,,,
4,000002b66c9c498e,/m/01g317,0.257812,0.346875,0.235938,0.385938,,,,,


In [ ]:
for i in range(len(class_strings)):
  filtered_class_data.loc[filtered_class_data['LabelName'] == class_strings[i], 'classNumber'] = i

#calculate center x and center y
filtered_class_data['center x'] = (filtered_class_data['XMin'] + filtered_class_data['XMax'])/2
filtered_class_data['center y'] = (filtered_class_data['YMin'] + filtered_class_data['YMax'])/2

#calculate width and height
filtered_class_data['width'] = (filtered_class_data['XMax'] - filtered_class_data['XMin'])
filtered_class_data['height'] = (filtered_class_data['YMax'] - filtered_class_data['YMin'])

#Yolo format
Yolo_values = filtered_class_data.loc[:, ['ImageID', 'classNumber', 'center x', 'center y', 'width', 'height']].copy()
Yolo_values.head()

,ImageID,classNumber,center x,center y,width,height
0,000002b66c9c498e,0,0.103906,0.367969,0.182812,0.439062
1,000002b66c9c498e,0,0.150782,0.831250,0.251563,0.234375
2,000002b66c9c498e,0,0.231249,0.394531,0.159375,0.392188
3,000002b66c9c498e,0,0.342969,0.788282,0.173438,0.273437
4,000002b66c9c498e,0,0.302343,0.310938,0.089063,0.150000


In [ ]:
image_path = '/content/drive/MyDrive/COLAB/YOLOv3/images_dataset/Person'
os.chdir(image_path)

for current_dir, dirs, files in os.walk('.'):
  for f in files:
    if f.endswith('.jpg'):
      image_title = f[:-4]
      Yolo_file = Yolo_values.loc[Yolo_values['ImageID'] == image_title]

      datafr = Yolo_file.loc[:, ['classNumber', 'center x', 'center y', 'width', 'height']].copy()

      save_path = image_path + '/' + image_title + '.txt'

      datafr.to_csv(save_path, header=False, index=False, sep=' ')
